In [27]:
# Packages
import numpy as np
import os
import cv2

import torch
from torchvision import models
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, TensorDataset, Dataset
import torchvision.transforms as transforms
import torch.optim as optim


from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# from PIL import Image


In [49]:
# Classes

class downNet1(nn.Module):
    def __init__(self):
        super(downNet1, self).__init__()

        # First 2D convolutional layer, taking in 1 input channel (image),
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv1 = nn.Conv2d(1, 64, 3, 1, padding=1)

        # Activation Function
        self.act1 = nn.ReLU()

        # Second 2D convolutional layer, taking in the 64 input layers,
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv2 = nn.Conv2d(64, 64, 3, 1, padding=1)

        # Activation Function
        self.act2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        output = x
        return output

    
class downNet2(nn.Module):
    def __init__(self):
        super(downNet2, self).__init__()
        
        self.conv1 = nn.Conv2d(64, 128, 3, 1, padding=1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(128, 128, 3, 1, padding=1)
        
        self.act2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        output = x
        return output
        
        
class downNet3(nn.Module):
    def __init__(self):
        super(downNet3, self).__init__()
        
        self.conv1 = nn.Conv2d(128, 256, 3, 1, padding=1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(256, 256, 3, 1, padding=1)
        
        self.act2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        output = x
        return output
        
        
class downNet4(nn.Module):
    def __init__(self):
        super(downNet4, self).__init__()
        
        self.conv1 = nn.Conv2d(256, 512, 3, 1, padding=1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(512, 512, 3, 1, padding=1)
        
        self.act2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        output = x
        return output
        
class upNet1(nn.Module):
    def __init__(self):
        super(upNet1, self).__init__()

        self.conv1 = nn.Conv2d(1024, 512, 3, 1, padding=1)

        self.act1 = nn.ReLU()

        self.conv2 = nn.Conv2d(512, 512, 3, 1, padding=1)

        self.act2 = nn.ReLU()
        
        self.upSamp = nn.ConvTranspose2d(512, 256, 2, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.upSamp(x)
        output = x
        return output

    
class upNet2(nn.Module):
    def __init__(self):
        super(upNet2, self).__init__()
        
        self.conv1 = nn.Conv2d(512, 256, 3, 1, padding=1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(256, 256, 3, 1, padding=1)
        
        self.act2 = nn.ReLU()
        
        self.upSamp = nn.ConvTranspose2d(256, 128, 2, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.upSamp(x)
        output = x
        return output
        
        
class upNet3(nn.Module):
    def __init__(self):
        super(upNet3, self).__init__()
        
        self.conv1 = nn.Conv2d(256, 128, 3, 1, padding=1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(128, 128, 3, 1, padding=1)
        
        self.act2 = nn.ReLU()
        
        self.upSamp = nn.ConvTranspose2d(128, 64, 2, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.upSamp(x)
        output = x
        return output
        
        
class upNet4(nn.Module):
    def __init__(self):
        super(upNet4, self).__init__()
        
        self.conv1 = nn.Conv2d(128, 64, 3, 1, padding=1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(64, 64, 3, 1, padding=1)
        
        self.act2 = nn.ReLU()
        
        # 2 classes, so final output should be 2
        self.conv3 = nn.Conv2d(64, 1, 1, 1) 
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.conv3(x)
        output = x
        return output
        
        
class bottleNeck(nn.Module):
    def __init__(self):
        super(bottleNeck, self).__init__()
        
        self.conv1 = nn.Conv2d(512, 1024, 3, 1, padding=1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(1024, 1024, 3, 1, padding=1)
        
        self.act2 = nn.ReLU()
        
        self.upSamp = nn.ConvTranspose2d(1024, 512, 2, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.upSamp(x)
        output = x
        return output
        
     
class pool(nn.Module):
    def __init__(self):
        super(pool, self).__init__()

        self.pooling = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        x = self.pooling(x)
        output = x
        return output
        
       
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        # Encoder (Downsampling)
        self.down1 = downNet1()
        self.pool1 = pool()
        
        self.down2 = downNet2()
        self.pool2 = pool()
        
        self.down3 = downNet3()
        self.pool3 = pool()
        
        self.down4 = downNet4()
        self.pool4 = pool()
        
        # Bottleneck
        self.bottleneck = bottleNeck()
        
        # Decoder (Upsampling)
        self.up1 = upNet1()
        self.up2 = upNet2()
        self.up3 = upNet3()
        self.up4 = upNet4()
        
    def copy_and_crop(self, upsampled, skip):
        # Crop the skip connection to match the upsampled dimensions
        _, _, H, W = upsampled.size()
        skip = skip[:, :, :H, :W]  # Crop the skip connection
        return torch.cat((upsampled, skip), dim=1) 
        
    def forward(self, x):
        # Downsampling path
        
        enc1 = self.down1(x)
        enc1_pool = self.pool1(enc1)
        
        enc2 = self.down2(enc1_pool)
        enc2_pool = self.pool2(enc2)
        
        enc3 = self.down3(enc2_pool)
        enc3_pool = self.pool3(enc3)
        
        enc4 = self.down4(enc3_pool)
        enc4_pool = self.pool4(enc4)
        
        # Bottleneck
        bottleneck_out = self.bottleneck(enc4_pool)
        
        # Upsampling path
        copy1 = self.copy_and_crop(bottleneck_out, enc4)
        dec1 = self.up1(copy1)
         
        copy2 = self.copy_and_crop(dec1, enc3)
        dec2 = self.up2(copy2)
        
        copy3 = self.copy_and_crop(dec2, enc2)  # Skip connection
        dec3 = self.up3(copy3)
        
        copy4 = self.copy_and_crop(dec3, enc1)  # Skip connection
        dec4 = self.up4(copy4)
        
        
        return dec4
    

# Transformer
class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim, num_heads, num_layers):
        super(TransformerEncoder, self).__init__()
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(embed_dim, num_heads)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        # x shape: (B, C, H, W) -> (H*W, B, C)
        B, C, H, W = x.size()
        x = x.permute(0, 2, 3, 1).reshape(-1, B, C)
        
        for layer in self.layers:
            x = layer(x)

        x = self.norm(x)
        x = x.view(H, W, B, C).permute(2, 3, 0, 1)  # (B, C, H, W)
        return x
    
    
class TransUNet(nn.Module):
    def __init__(self):
        super(TransUNet, self).__init__()

        # Encoder (Downsampling)
        self.down1 = downNet1()
        self.pool1 = pool()
        
        self.down2 = downNet2()
        self.pool2 = pool()
        
        self.down3 = downNet3()
        self.pool3 = pool()
        
        self.down4 = downNet4()
        self.pool4 = pool()
        
        # Bottleneck
        self.bottleneck = bottleNeck()

        # Transformer Encoder
        self.transformer = TransformerEncoder(embed_dim=512, num_heads=8, num_layers=4)

        # Decoder (Upsampling)
        self.up1 = upNet1()
        self.up2 = upNet2()
        self.up3 = upNet3()
        self.up4 = upNet4()

    def copy_and_crop(self, upsampled, skip):
        # Crop the skip connection to match the upsampled dimensions
        _, _, H, W = upsampled.size()
        skip = skip[:, :, :H, :W]  # Crop the skip connection
        return torch.cat((upsampled, skip), dim=1) 
        
    def forward(self, x):
        # Downsampling path
        
        enc1 = self.down1(x)
        enc1_pool = self.pool1(enc1)
        
        enc2 = self.down2(enc1_pool)
        enc2_pool = self.pool2(enc2)
        
        enc3 = self.down3(enc2_pool)
        enc3_pool = self.pool3(enc3)
        
        enc4 = self.down4(enc3_pool)
        enc4_pool = self.pool4(enc4)
        
        # Bottleneck
        bottleneck_out = self.bottleneck(enc4_pool)

        # Apply Transformer
        transformer_out = self.transformer(bottleneck_out)

        # Upsampling path
        copy1 = self.copy_and_crop(transformer_out, enc4)
        dec1 = self.up1(copy1)
         
        copy2 = self.copy_and_crop(dec1, enc3)
        dec2 = self.up2(copy2)
        
        copy3 = self.copy_and_crop(dec2, enc2)  # Skip connection
        dec3 = self.up3(copy3)
        
        copy4 = self.copy_and_crop(dec3, enc1)  # Skip connection
        dec4 = self.up4(copy4)
        
        return dec4

    
    
# Dice Coefficient
def dice_coefficient(pred, target, smooth=1e-6):
    pred = pred.view(-1)
    target = target.view(-1)
    
    intersection = (pred * target).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    return dice

my_nn = TransUNet()
# print(my_nn)

In [50]:
# Run a test
random_data = torch.rand((1, 1, 224, 224))

result = my_nn(random_data)
print(result[:,0,:,:])

tensor([[[0.0704, 0.0713, 0.0661,  ..., 0.0701, 0.0669, 0.0745],
         [0.0724, 0.0714, 0.0711,  ..., 0.0694, 0.0722, 0.0796],
         [0.0677, 0.0656, 0.0691,  ..., 0.0679, 0.0731, 0.0765],
         ...,
         [0.0730, 0.0725, 0.0725,  ..., 0.0722, 0.0760, 0.0763],
         [0.0709, 0.0717, 0.0727,  ..., 0.0717, 0.0784, 0.0739],
         [0.0756, 0.0761, 0.0755,  ..., 0.0781, 0.0788, 0.0750]]],
       grad_fn=<SliceBackward0>)


In [33]:
# Classes to load in data

class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.image_filenames = os.listdir(images_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # Load the image
        img_path = os.path.join(self.images_dir, self.image_filenames[idx])
        image = cv2.imread(img_path)  # Load image
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Load the corresponding mask
        mask_path = os.path.join(self.masks_dir, self.image_filenames[idx])  # Assuming masks have the same filename
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        # Ensure mask is of type long for segmentation
        mask = mask.long()  

        return image, mask


class MultiDataset(Dataset):
    def __init__(self, image_dirs, mask_dirs, transform=None):
        self.image_dirs = image_dirs
        self.mask_dirs = mask_dirs
        self.transform = transform
        self.image_paths = []
        self.mask_paths = []
        
        # Collect image and mask paths
        for img_dir, mask_dir in zip(image_dirs, mask_dirs):
            img_files = os.listdir(img_dir)
            for img_file in img_files:
                img_path = os.path.join(img_dir, img_file)
                mask_path = os.path.join(mask_dir, img_file)  # Assume the same naming
                if os.path.exists(mask_path):  # Ensure mask exists
                    self.image_paths.append(img_path)
                    self.mask_paths.append(mask_path)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        
        # Load images and masks
        image = cv2.imread(img_path)  # Load image
        
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
            
        # Ensure mask is of type long for segmentation
        # mask = mask.long()

        return image, mask





In [43]:
# Load the data

dataPath = [
    'ImagesAndMasks/AllCovid/Covid',
    'ImagesAndMasks/AllNormal/Normal',
    'ImagesAndMasks/AllOpacity/Lung_Opacity',
    'ImagesAndMasks/AllPneumonia/Viral_Pneumonia'   
]

maskPath = [
    'ImagesAndMasks/AllCovid/COVIDmasks',
    'ImagesAndMasks/AllNormal/Normalmasks',
    'ImagesAndMasks/AllOpacity/Lung_Opacitymasks',
    'ImagesAndMasks/AllPneumonia/Viral_Pneumoniamasks'
]


# Define your transformations
transform = transforms.Compose([
    transforms.ToPILImage(),               # Convert to PIL Image first
    transforms.Resize((224, 224)),         # Resize to 224x224
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),                  # Convert image to tensor
])


#
dataset = MultiDataset(dataPath, maskPath, transform=transform)

# Split into train, test, val
trainRatio = 0.75
valRatio = 0.1
testRatio = 1 - trainRatio - valRatio

# Calculate the sizes of each split
train_size = int(trainRatio * len(dataset))
val_size = int(valRatio * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


# # Create the DataLoader
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
# print(dataloader.mask)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

TypeError: 'DataLoader' object is not subscriptable

In [45]:
# Initialize model, loss function, and optimizer
model = UNet()

model.to(device) 

criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Adjust learning rate as needed

# Training loop
num_epochs = 10  # Set the number of epochs
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)
        images = images.float()  # Convert to FloatTensor
        masks = masks.float()     # Convert to FloatTensor
        # print(f"Images type: {images.dtype}, shape: {images.shape}")
        # print(f"Masks type: {masks.dtype}, shape: {masks.shape}")
        # print(images)
        
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        # print(f"Output shape: {outputs.shape}")

        # Calculate loss
        loss = criterion(outputs, masks)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')
    
    
    # Set the model to evaluation mode
    model.eval()
    
    val_loss = 0.0
    
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)
            images = images.float()  # Convert to FloatTensor
            masks = masks.float()
            
            # print(images.shape)
            # print(masks.shape)

            outputs = model(images)
            
            loss = criterion(outputs, masks)
            val_loss += loss.item()
            
    
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
    

print('Training complete.')

Epoch [1/10], Loss: 0.1892
Validation Loss: 0.0886
Epoch [2/10], Loss: 0.0461
Validation Loss: 0.0492
Epoch [3/10], Loss: 0.0314
Validation Loss: 0.0378
Epoch [4/10], Loss: 0.0250
Validation Loss: 0.0311
Epoch [5/10], Loss: 0.0216
Validation Loss: 0.0319
Epoch [6/10], Loss: 0.0204
Validation Loss: 0.0287
Epoch [7/10], Loss: 0.0181
Validation Loss: 0.0246
Epoch [8/10], Loss: 0.0171
Validation Loss: 0.0346
Epoch [9/10], Loss: 0.0166
Validation Loss: 0.0242
Epoch [10/10], Loss: 0.0151
Validation Loss: 0.0234
Training complete.
